# Vehicle routing with Timefold in a Kotlin notebook

This notebook solves a simple Capacitated Vehicle Routing Problem (CVRP) in Kotlin with Timefold, the open source solver AI.

Input:
* A set of visits with a location and a load
* A set of vehicles with a home location and a capacity

Output:
* Each visit assigned to a vehicle
* Per vehicle the order in which to travel to the visits assigned to it

Constraints:
* Hard: Do not exceed the capacity of each visit.
* Soft: Minimize the travel distance.

## Dependencies

Let's use Timefold to optimize the vehicle routing problem and jackson to read the input JSON file:

In [77]:
@file:DependsOn("ai.timefold.solver:timefold-solver-core:1.11.0")
@file:DependsOn("com.fasterxml.jackson.module:jackson-module-kotlin:2.17.1")

## Data classes

### Location

A location is a point on the earth, specified by a latitude and a longitude:

In [78]:
import com.fasterxml.jackson.annotation.JsonFormat

@JsonFormat(shape = JsonFormat.Shape.ARRAY)
data class Location(
    val latitude: Double,
    val longitude: Double) {
    
    fun calcEuclideanDistanceTo(other: Location): Double {
        val xDifference = latitude - other.latitude
        val yDifference = longitude - other.longitude
        return Math.sqrt(xDifference * xDifference + yDifference * yDifference)
    }
    
}

### Visit



In [79]:
import java.time.Duration
import java.time.LocalDateTime
import com.fasterxml.jackson.annotation.JsonFormat

data class Visit(
    val name: String,
    val location: Location,
    val load: Int,
    val minStartTime: LocalDateTime,
    val maxEndTime: LocalDateTime,
    val serviceDuration: Duration) {

    // Fonction pour calculer l'heure de départ en fonction de l'heure d'arrivée et de la durée du service
    fun getDepartureTime(arrivalTime: LocalDateTime): LocalDateTime {
        return arrivalTime.plus(serviceDuration)
    }

    override fun toString(): String = name
}


### Vehicle



In [80]:
import ai.timefold.solver.core.api.domain.entity.PlanningEntity
import ai.timefold.solver.core.api.domain.variable.PlanningListVariable
import java.time.LocalDateTime

@PlanningEntity
data class Vehicle(
    val name: String,
    val homeLocation: Location,
    val capacity: Int,
    val departureTime: LocalDateTime) {

    @PlanningListVariable
    var visits: MutableList<Visit> = ArrayList()

    // Constructeur sans argument requis pour Timefold
    constructor() : this("", Location(0.0, 0.0), 0, LocalDateTime.now())

    override fun toString(): String = name
}


## Constraints

There are hard and soft constraints:

In [81]:
import ai.timefold.solver.core.api.score.buildin.hardsoftlong.HardSoftLongScore
import ai.timefold.solver.core.api.score.stream.Constraint
import ai.timefold.solver.core.api.score.stream.ConstraintFactory
import ai.timefold.solver.core.api.score.stream.ConstraintProvider
import java.time.Duration
import java.time.LocalDateTime

class VehicleRoutingConstraintProvider : ConstraintProvider {

    override fun defineConstraints(constraintFactory: ConstraintFactory): Array<Constraint>? {
        return arrayOf(
            // Contraintes dures
            capacity(constraintFactory),
            timeWindows(constraintFactory),
            // Contraintes souples
            minimizeDistance(constraintFactory)
        )
    }

    fun capacity(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory
            .forEach(Vehicle::class.java)
            .expand({ vehicle -> vehicle.visits.sumOf { it.load } })
            .filter({ vehicle, load -> load > vehicle.capacity })
            .penalizeLong(HardSoftLongScore.ONE_HARD,
                { vehicle, load -> (load - vehicle.capacity).toLong() })
            .asConstraint("vehicle-routing", "Capacity");
    }

    fun timeWindows(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory
            .forEach(Visit::class.java)
            .filter({ visit ->
                val arrivalTime = calculateArrivalTime(visit)
                arrivalTime.isBefore(visit.minStartTime) || visit.getDepartureTime(arrivalTime).isAfter(visit.maxEndTime)
            })
            .penalizeLong(HardSoftLongScore.ONE_HARD, { visit ->
                val arrivalTime = calculateArrivalTime(visit)
                val penalty = if (arrivalTime.isBefore(visit.minStartTime)) {
                    Duration.between(arrivalTime, visit.minStartTime).seconds
                } else {
                    Duration.between(visit.getDepartureTime(arrivalTime), visit.maxEndTime).seconds
                }
                penalty
            })
            .asConstraint("vehicle-routing", "Time Windows")
    }

    fun minimizeDistance(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory
            .forEach(Vehicle::class.java)
            .penalizeLong(HardSoftLongScore.ONE_SOFT, { vehicle ->
                var distance: Double = 0.0
                var previousLocation: Location = vehicle.homeLocation
                for (visit in vehicle.visits) {
                    distance += previousLocation.calcEuclideanDistanceTo(visit.location)
                    previousLocation = visit.location
                }
                distance += previousLocation.calcEuclideanDistanceTo(vehicle.homeLocation)
                (distance * 1_000_000.0).toLong()
            })
            .asConstraint("vehicle-routing", "Minimize distance");
    }

    // Fonction pour calculer l'heure d'arrivée prévue à chaque visite
    fun calculateArrivalTime(visit: Visit): LocalDateTime {
        var arrivalTime = visit.minStartTime
        val previousVisit = findPreviousVisit(visit)
        if (previousVisit != null) {
            val previousDepartureTime = calculateDepartureTime(previousVisit)
            val travelTime = previousVisit.location.calcEuclideanDistanceTo(visit.location) / AVERAGE_SPEED_KMH * 3600 // en secondes
            arrivalTime = previousDepartureTime.plusSeconds(travelTime.toLong())
        } else if (visit.vehicle != null) {
            // Première visite du véhicule
            val departureTime = visit.vehicle.departureTime
            val travelTime = visit.vehicle.homeLocation.calcEuclideanDistanceTo(visit.location) / AVERAGE_SPEED_KMH * 3600 // en secondes
            arrivalTime = departureTime.plusSeconds(travelTime.toLong())
        }
        return arrivalTime
    }

    fun calculateDepartureTime(visit: Visit): LocalDateTime {
        val arrivalTime = calculateArrivalTime(visit)
        val startServiceTime = if (arrivalTime.isBefore(visit.minStartTime)) visit.minStartTime else arrivalTime
        return startServiceTime.plus(visit.serviceDuration)
    }

    // Fonction pour trouver la visite précédente dans la tournée
    fun findPreviousVisit(visit: Visit): Visit? {
        val vehicle = visit.vehicle ?: return null
        val index = vehicle.visits.indexOf(visit)
        return if (index > 0) vehicle.visits[index - 1] else null
    }

    companion object {
        const val AVERAGE_SPEED_KMH = 50.0 // Vitesse moyenne supposée
    }

}


Line_124.jupyter.kts (73:26 - 33) Unresolved reference: vehicle
Line_124.jupyter.kts (75:39 - 46) Unresolved reference: vehicle
Line_124.jupyter.kts (76:36 - 43) Unresolved reference: vehicle
Line_124.jupyter.kts (90:29 - 36) Unresolved reference: vehicle

## Schedule

The `Schedule` class holds the entire dataset.
It contains the list of all vehicles (the entities the solver must fill in) and a list of all visits (the values it needs to assign to those entities).

In [82]:
import ai.timefold.solver.core.api.domain.solution.PlanningEntityCollectionProperty
import ai.timefold.solver.core.api.domain.solution.PlanningScore
import ai.timefold.solver.core.api.domain.solution.PlanningSolution
import ai.timefold.solver.core.api.domain.solution.ProblemFactCollectionProperty
import ai.timefold.solver.core.api.domain.valuerange.ValueRangeProvider
import ai.timefold.solver.core.api.score.buildin.hardsoftlong.HardSoftLongScore


@PlanningSolution
data class Schedule(
    val name: String,
    @PlanningEntityCollectionProperty
    val vehicles: List<Vehicle>,
    @ProblemFactCollectionProperty
    @ValueRangeProvider
    val visits: List<Visit>) {

    @PlanningScore
    var score: HardSoftLongScore? = null

    // No-arg constructor required for Timefold
    constructor() : this("", emptyList(), emptyList())

}

## Read the input data

Read the input dataset from a JSON file into a `Schedule` instance:

In [83]:
import java.io.File
import com.fasterxml.jackson.module.kotlin.jacksonObjectMapper
import com.fasterxml.jackson.module.kotlin.readValue
import java.time.LocalDateTime
import java.time.format.DateTimeFormatter

val mapper = jacksonObjectMapper()

// Configurer le format de date/heure pour le parseur
val dateTimeFormatter = DateTimeFormatter.ISO_DATE_TIME

mapper.findAndRegisterModules()

val problem: Schedule = mapper.readValue(File("vehicle-routing-data.json"))


vehicle-routing-data.json (The system cannot find the file specified)
java.io.FileNotFoundException: vehicle-routing-data.json (The system cannot find the file specified)
	at java.base/java.io.FileInputStream.open0(Native Method)
	at java.base/java.io.FileInputStream.open(FileInputStream.java:213)
	at java.base/java.io.FileInputStream.<init>(FileInputStream.java:152)
	at com.fasterxml.jackson.core.TokenStreamFactory._fileInputStream(TokenStreamFactory.java:318)
	at com.fasterxml.jackson.core.JsonFactory.createParser(JsonFactory.java:1219)
	at com.fasterxml.jackson.databind.ObjectMapper.readValue(ObjectMapper.java:3733)
	at Line_126_jupyter.<init>(Line_126.jupyter.kts:17)
	at java.base/jdk.internal.reflect.DirectConstructorHandleAccessor.newInstance(DirectConstructorHandleAccessor.java:62)
	at java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:502)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:486)
	at kotlin.script.experimental.jvm

## Solve

Let's solve for 30 seconds:

In [84]:
import ai.timefold.solver.core.config.solver.SolverConfig
import ai.timefold.solver.core.api.solver.SolverFactory
import ai.timefold.solver.core.api.solver.Solver

val solverFactory: SolverFactory<Schedule> = SolverFactory.create(SolverConfig()
    .withSolutionClass(Schedule::class.java)
    .withEntityClasses(Vehicle::class.java)
    .withConstraintProviderClass(VehicleRoutingConstraintProvider::class.java)
    // Le solver fonctionne pendant 30 secondes sur ce jeu de données
    .withTerminationSpentLimit(Duration.ofSeconds(30)))

println("Résolution du problème...")
val solver: Solver<Schedule> = solverFactory.buildSolver()
val solution: Schedule = solver.solve(problem)
println("Résolution terminée avec un score de (${solution.score}).")


Line_127.jupyter.kts (8:34 - 66) Unresolved reference: VehicleRoutingConstraintProvider

## Print the schedule

Print the visits per vehicle:

In [85]:
HTML(buildString {
    append("<p style='font-size: x-large'>Score: ${solution.score}</p>")
    append("<ul>")
    for (vehicle in solution.vehicles) {
        append("<li>${vehicle.name}: ${vehicle.visits.joinToString(", ")}</li>")
    }
    append("</ul>")
})

Line_128.jupyter.kts (2:52 - 60) Unresolved reference: solution
Line_128.jupyter.kts (4:21 - 29) Unresolved reference: solution

## Visualization

Visualize the solution:

In [86]:
%use lets-plot
%use lets-plot-gt

### Map

In [87]:
val locations = mutableListOf<Location>()
solution.vehicles.forEach { vehicle ->
    locations.add(vehicle.homeLocation)
    locations.addAll(vehicle.visits.map { it.location })
    locations.add(vehicle.homeLocation)
}

val dataset = mapOf(
        "latitude" to locations.map { it.latitude },
        "longitude" to locations.map { it.longitude },
)

print("The notebook must be trusted for the map to render.")
letsPlot(dataset) + geomPath() { x = "longitude"; y = "latitude" }

Line_138.jupyter.kts (2:1 - 9) Unresolved reference: solution
Line_138.jupyter.kts (2:19 - 26) Overload resolution ambiguity: 
public inline fun <T> Iterable<TypeVariable(T)>.forEach(action: (TypeVariable(T)) -> Unit): Unit defined in kotlin.collections
public inline fun <K, V> Map<out TypeVariable(K), TypeVariable(V)>.forEach(action: (Map.Entry<TypeVariable(K), TypeVariable(V)>) -> Unit): Unit defined in kotlin.collections
Line_138.jupyter.kts (2:29 - 36) Cannot infer a type for this parameter. Please specify it explicitly.
Line_138.jupyter.kts (4:43 - 45) Unresolved reference: it

## Statistics

For a big dataset, a schedule visualization is often too verbose.
Let's visualize the solution through statistics:

In [88]:
%use kandy

### Visits per vehicle

In [89]:
val vehicles = solution.vehicles.map { it.name }
val visitCounts = solution.vehicles.map { it.visits.size }

plot {
    layout.title = "Visits per vehicle"
    bars {
        x(vehicles) { axis.name = "Vehicle" }
        y(visitCounts) { axis.name = "Visits" }
    }
}

Line_140.jupyter.kts (1:16 - 24) Unresolved reference: solution
Line_140.jupyter.kts (1:40 - 42) Unresolved reference: it
Line_140.jupyter.kts (2:19 - 27) Unresolved reference: solution
Line_140.jupyter.kts (2:43 - 45) Unresolved reference: it

### Load per vehicle

In [90]:
val vehicles = solution.vehicles.map { it.name }
val load = solution.vehicles.map { it.visits.sumOf { it.load } }

plot {
    layout.title = "Load per vehicle"
    bars {
        x(vehicles) { axis.name = "Vehicle" }
        y(load) { axis.name = "Load" }
    }
}

Line_141.jupyter.kts (1:16 - 24) Unresolved reference: solution
Line_141.jupyter.kts (1:40 - 42) Unresolved reference: it
Line_141.jupyter.kts (2:12 - 20) Unresolved reference: solution
Line_141.jupyter.kts (2:36 - 38) Unresolved reference: it
Line_141.jupyter.kts (2:54 - 56) Unresolved reference: it

## Analyze the score

Break down the score per constraint and print it:

In [91]:
import ai.timefold.solver.core.api.solver.SolutionManager

val solutionManager = SolutionManager.create(solverFactory)
val scoreAnalysis = solutionManager.analyze(solution)

HTML(buildString {
    append("<p style='font-size: x-large'>Score: ${scoreAnalysis.score}</p>")
    append("<ul>")
    for (constraint in scoreAnalysis.constraintMap().values) {
        append("<li>${constraint.constraintRef().constraintName}: ${constraint.score.toShortString()}</li>")
    }
    append("</ul>")
})

Line_142.jupyter.kts (3:46 - 59) Unresolved reference: solverFactory
Line_142.jupyter.kts (4:45 - 53) Unresolved reference: solution

## Conclusion

To learn more about planning optimization, visit [timefold.ai](https://timefold.ai).